In [92]:
from sympy import *
k_iso, theta, delta =  symbols("k_iso, theta, delta")
Ex, Ey =  symbols("Ex, Ey")

In [93]:
Q      = Matrix([[cos(theta), sin(theta)],[-sin(theta), cos(theta)]]) 
E_cart = Matrix([[Ex],[Ey]]) 

In [94]:
# Transform E to material coordinates
E_rot  = Q*E_cart
E_rot

Matrix([
[ Ex*cos(theta) + Ey*sin(theta)],
[-Ex*sin(theta) + Ey*cos(theta)]])

In [95]:
# Flux in material coordinates (transverse)
K     = Matrix([[k_iso, 0],[0, k_iso/delta]])
q_rot = -K *E_rot
q_rot

Matrix([
[       -k_iso*(Ex*cos(theta) + Ey*sin(theta))],
[-k_iso*(-Ex*sin(theta) + Ey*cos(theta))/delta]])

In [96]:
# Transform back to Cartesian
q_cart = Q.transpose() * q_rot
q_cart

Matrix([
[-k_iso*(Ex*cos(theta) + Ey*sin(theta))*cos(theta) + k_iso*(-Ex*sin(theta) + Ey*cos(theta))*sin(theta)/delta],
[-k_iso*(Ex*cos(theta) + Ey*sin(theta))*sin(theta) - k_iso*(-Ex*sin(theta) + Ey*cos(theta))*cos(theta)/delta]])

In [97]:
# Cartesian conductivity tensor
kxx = -q_cart[0].diff(Ex)
kxy = -q_cart[0].diff(Ey)
kyx = -q_cart[1].diff(Ex)
kyy = -q_cart[1].diff(Ey)
print(julia_code(kxx))
print(julia_code(kxy))
print(julia_code(kyx))
print(julia_code(kyy))

k_iso .* cos(theta) .^ 2 + k_iso .* sin(theta) .^ 2 ./ delta
k_iso .* sin(theta) .* cos(theta) - k_iso .* sin(theta) .* cos(theta) ./ delta
k_iso .* sin(theta) .* cos(theta) - k_iso .* sin(theta) .* cos(theta) ./ delta
k_iso .* sin(theta) .^ 2 + k_iso .* cos(theta) .^ 2 ./ delta


In [98]:
def stencil(kxxW, kxyW, kxxE, kxyE,   kyyS, kyxS, kyyN, kyxN,     kxySW, kxySE, kxyNW, kxyNE, kxxSW, kxxSE, kxxNW, kxxNE, kyySW, kyySE, kyyNW, kyyNE,     dxdXN, dxdXS, dydXW, dydXE, dxdΥN, dxdΥS, dydΥW, dydΥE,    dxdXSW, dxdXSE, dxdXNW, dxdXNE, dydXSW, dydXSE, dydXNW, dydXNE, dxdYSW, dxdYSE, dxdYNW, dxdYNE, dydYSW, dydYSE, dydYNW, dydYNE ):
    uSW, uS, uSE, uW, uC, uE, uNW, uN, uNE = symbols('uSW, uS, uSE, uW, uC, uE, uNW, uN, uNE')

    dx, dy = symbols('dx,dy')

    dy = dx

    ExSW = (uS-uSW)/dx 
    ExW  = (uC-uW)/dx 
    ExNW = (uN-uNW)/dx 
    ExSE = (uSE-uS)/dx 
    ExE  = (uE-uC)/dx 
    ExNE = (uNE-uN)/dx 

    EySW = (uW-uSW)/dy 
    EyS  = (uC-uS)/dy 
    EyNW = (uNW-uW)/dy 
    EySE = (uE-uSE)/dy 
    EyN  = (uN-uC)/dy 
    EyNE = (uNE-uE)/dy

    # ExS = Rational(1,4)*(ExW + ExE + ExSW + ExSE)
    # ExN = Rational(1,4)*(ExW + ExE + ExNW + ExNE)
    # EyW = Rational(1,4)*(EyS + EyN + EySW + EyNW)
    # EyE = Rational(1,4)*(EyS + EyN + EySE + EyNE)
    
    # qxW = -kxxW*ExW -kxyW*EyW
    # qxE = -kxxE*ExE -kxyE*EyE
    # qyS = -kyyS*EyS -kyxS*ExS
    # qyN = -kyyN*EyN -kyxN*ExN

    # kxyS = kyxS
    # kxyN = kyxN
    # qxW = -kxxW*ExW - 0*Rational(1,4)*(kxyS*EyS  + kxyN*EyN + kxySW*EySW + kxyNW*EyN)
    # qxE = -kxxE*ExE - 0*Rational(1,4)*(kxyS*EyS  + kxyN*EyN + kxySE*EySE + kxyNE*EyN)
    # qyS = -kyyS*EyS - 0*Rational(1,4)*(kxyW*ExW  + kxyE*ExE + kxySW*ExSW + kxySE*ExS)
    # qyN = -kyyN*EyN - 0*Rational(1,4)*(kxyW*ExW  + kxyE*ExE + kxyNW*ExNW + kxyNE*ExN)

    DxNE = 1/2*(  (uNE+uE) - (uN+uC)) / dx
    DyNE = 1/2*(  (uNE+uN) - (uE+uC)) / dy
    DxNW = 1/2*( -(uNW+uW) + (uN+uC)) / dx
    DyNW = 1/2*(  (uNW+uN) - (uW+uC)) / dy
    DxSE = 1/2*(  (uSE+uE) - (uS+uC)) / dx
    DySE = 1/2*( -(uSE+uS) + (uE+uC)) / dy
    DxSW = 1/2*( -(uSW+uW) + (uS+uC)) / dx
    DySW = 1/2*( -(uSW+uS) + (uW+uC)) / dy

    qxSW = - kxxSW*DxSW - 0*kxySW*DySW
    qxSE = - kxxSE*DxSE - 0*kxySE*DySE
    qxNW = - kxxNW*DxNW - 0*kxyNW*DyNW
    qxNE = - kxxNE*DxNE - 0*kxyNE*DyNE

    qySW = - 0*kxySW*DxSW - kyySW*DySW
    qySE = - 0*kxySE*DxSE - kyySE*DySE
    qyNW = - 0*kxyNW*DxNW - kyyNW*DyNW
    qyNE = - 0*kxyNE*DxNE - kyyNE*DyNE

    dqxdxN = (qxNE - qxNW) / dx * dxdXN 
    dqxdxS = (qxSE - qxSW) / dx * dxdXS
    dqxdyW = (qxNW - qxSW) / dy * dydXW
    dqxdyE = (qxNE - qxSE) / dy * dydXE

    dqydxN = (qyNE - qyNW) / dx * dxdΥN
    dqydxS = (qySE - qySW) / dx * dxdΥS
    dqydyW = (qyNW - qySW) / dy * dydΥW
    dqydyE = (qyNE - qySE) / dy * dydΥE

    # dqxdxN = (qxNE - qxNW) / dx * 1/2*(dxdXNE+dxdXNW)
    # dqxdxS = (qxSE - qxSW) / dx * 1/2*(dxdXSE+dxdXSW)
    # dqxdyW = (qxNW - qxSW) / dy * 1/2*(dydXNW+dydXSW)
    # dqxdyE = (qxNE - qxSE) / dy * 1/2*(dydXNE+dydXSE)

    # dqydxN = (qyNE - qyNW) / dx * 1/2*(dxdYNE+dxdYNW)
    # dqydxS = (qySE - qySW) / dx * 1/2*(dxdYSE+dxdYSW)
    # dqydyW = (qyNW - qySW) / dy * 1/2*(dydYNW+dydYSW)
    # dqydyE = (qyNE - qySE) / dy * 1/2*(dydYNE+dydYSE)

    f  = -(dqxdxN + dqxdxS)/2 
    f += -(dqxdyW + dqxdyE)/2 
    f += -(dqydxN + dqydxS)/2 
    f += -(dqydyW + dqydyE)/2 

    # qxW = -1/2*( kxxSW*DxSW + kxxNW*DxNW + kxySW*DySW + kxyNW*DyNW )
    # qxE = -1/2*( kxxSE*DxSE + kxxNE*DxNE + kxySE*DySE + kxyNE*DyNE )
    # qyS = -1/2*( kxySW*DxSW + kxySE*DxSE + kyySW*DySW + kyySE*DySE )
    # qyN = -1/2*( kxyNW*DxNW + kxyNE*DxNE + kyyNW*DyNW + kyyNE*DyNE )

    # f  = -(qxSE + qxNE - qxSW - qxNW)/2/dx * dxdX
    # f += -(qxNE + qxNW - qxSW - qxSE)/2/dy * dydX
    # f += -(qySE + qyNE - qySW - qyNW)/2/dx * dxdY
    # f += -(qyNE + qyNW - qySW - qySE)/2/dy * dydY

    # dxdXSW,dxdXSE,dxdXNW,dxdXNE = symbols('dxdXSW,dxdXSE,dxdXNW,dxdXNE')
    # dydXSW,dydXSE,dydXNW,dydXNE = symbols('dydXSW,dydXSE,dydXNW,dydXNE')
    # dxdYSW,dxdYSE,dxdYNW,dxdYNE = symbols('dxdYSW,dxdYSE,dxdYNW,dxdYNE')
    # dydYSW,dydYSE,dydYNW,dydYNE = symbols('dydYSW,dydYSE,dydYNW,dydYNE')
    # dxdX, dydX, dxdY, dydY = symbols('dxdX, dydX, dxdY, dydY')

    f  = -(qxSE + qxNE - qxSW - qxNW)/2/dx * 1/4*(dxdXSW+dxdXSE+dxdXNW+dxdXNE)
    f += -(qxNE + qxNW - qxSW - qxSE)/2/dy * 1/4*(dydXSW+dydXSE+dydXNW+dydXNE)
    f += -(qySE + qyNE - qySW - qyNW)/2/dx * 1/4*(dxdYSW+dxdYSE+dxdYNW+dxdYNE)
    f += -(qyNE + qyNW - qySW - qySE)/2/dy * 1/4*(dydYSW+dydYSE+dydYNW+dydYNE)
    
    return  f # -(qxE - qxW)/dx -(qyN - qyS)/dy 

In [99]:
kxxW, kxyW, kxxE, kxyE = symbols('kxxW, kxyW, kxxE, kxyE')
kyyS, kyyN, kyxS, kyxN = symbols('kyyS, kyyN, kyxS, kyxN')
kxx1, kxy1, kyx1, kyy1 = symbols('kxx1, kxy1 kyx1, kyy1')
kxx2, kxy2, kyx2, kyy2 = symbols('kxx2, kxy2, kyx2, kyy2')

dxdXN, dxdXS, dydXW, dydXE, dxdΥN, dxdΥS, dydΥW, dydΥE = symbols('dxdXN, dxdXS, dydXW, dydXE, dxdΥN, dxdΥS, dydΥW, dydΥE')
dxdXSW, dxdXSE, dxdXNW, dxdXNE, dydXSW, dydXSE, dydXNW, dydXNE, dxdYSW, dxdYSE, dxdYNW, dxdYNE, dydYSW, dydYSE, dydYNW, dydYNE = symbols('dxdXSW, dxdXSE, dxdXNW, dxdXNE, dydXSW, dydXSE, dydXNW, dydXNE, dxdYSW, dxdYSE, dxdYNW, dxdYNE, dydYSW, dydYSE, dydYNW, dydYNE')

kxySW, kxySE, kxyNW, kxyNE = symbols('kxySW, kxySE, kxyNW, kxyNE')
kxxSW, kxxSE, kxxNW, kxxNE = symbols('kxxSW, kxxSE, kxxNW, kxxNE')
kyySW, kyySE, kyyNW, kyyNE = symbols('kyySW, kyySE, kyyNW, kyyNE')
uSW, uS, uSE, uW, uC, uE, uNW, uN, uNE = symbols('uSW, uS, uSE, uW, uC, uE, uNW, uN, uNE')


In [100]:
# dxdXN = 1
# dxdXS = 1 
# dydXW = 0 
# dydXE = 0 
# dxdΥN = 0
# dxdΥS = 0 
# dydΥW = 1 
# dydΥE = 1

In [101]:

# N/S
nodeN = stencil(kxxW, kxyW, kxxE, kxyE,  kyyS, kyxS, kyyN, kyxN,       kxy1, kxy1, kxy2, kxy2, kxx1, kxx1, kxx2, kxx2, kyy1, kyy1, kyy2, kyy2,  dxdXN, 3, dydXW, dydXE, dxdΥN, 2, dydΥW, dydΥE,         2,      2, dxdXNW, dxdXNE,      2,      2, dydXNW, dydXNE,      2,      2, dxdYNW, dxdYNE,      2,      2, dydYNW, dydYNE)
nodeS = stencil(kxxW, kxyW, kxxE, kxyE,  kyyS, kyxS, kyyN, kyxN,       kxy2, kxy2, kxy1, kxy1, kxx2, kxx2, kxx1, kxx1, kyy2, kyy2, kyy1, kyy1,  3, dxdXS, dydXW, dydXE, 2, dxdΥS, dydΥW, dydΥE,    dxdXSW, dxdXSE,      2,      2, dydXSW, dydXSE,      2,      2, dxdYSW, dxdYSE,      2,      2, dydYSW, dydYSE,      2,      2)
a=nodeS.diff(uN)
b=nodeN.diff(uS)
(a-b).simplify()

0.125*(kxx1*(dxdXNE + dxdXNW + 4) - kxx1*(dxdXSE + dxdXSW + 4) - kyy1*(dydYNE + dydYNW + 4) + kyy1*(dydYSE + dydYSW + 4))/dx**2

In [102]:
# W/E
nodeE = stencil(kxxW, kxyW, kxxE, kxyE,  kyyS, kyxS, kyyN, kyxN,       kxy1, kxy2, kxy1, kxy2, kxx1, kxx2, kxx1, kxx2, kyy1, kyy2, kyy1, kyy2,    dxdXN, dxdXS, dydXW, dydXE, dxdΥN, dxdΥS, dydΥW, dydΥE,    dxdXSW, dxdXSE, dxdXNW, dxdXNE, dydXSW, dydXSE, dydXNW, dydXNE, dxdYSW, dxdYSE, dxdYNW, dxdYNE, dydYSW, dydYSE, dydYNW, dydYNE )
nodeW = stencil(kxxW, kxyW, kxxE, kxyE,  kyyS, kyxS, kyyN, kyxN,       kxy2, kxy1, kxy2, kxy1, kxx2, kxx1, kxx2, kxx1, kyy2, kyy1, kyy2, kyy1,    dxdXN, dxdXS, dydXW, dydXE, dxdΥN, dxdΥS, dydΥW, dydΥE,    dxdXSW, dxdXSE, dxdXNW, dxdXNE, dydXSW, dydXSE, dydXNW, dydXNE, dxdYSW, dxdYSE, dxdYNW, dxdYNE, dydYSW, dydYSE, dydYNW, dydYNE )
a=nodeE.diff(uW)
b=nodeW.diff(uE)
(a-b).simplify()

0

In [103]:
# NE/SW
nodeNE = stencil(kxxW, kxyW, kxxE, kxyE,  kyyS, kyxS, kyyN, kyxN,       kxy1, kxy2, kxy2, kxy2, kxx1, kxx2, kxx2, kxx2, kyy1, kyy2, kyy2, kyy2,    dxdXN, dxdXS, dydXW, dydXE, dxdΥN, dxdΥS, dydΥW, dydΥE,   dxdXSW, dxdXSE, dxdXNW, dxdXNE, dydXSW, dydXSE, dydXNW, dydXNE, dxdYSW, dxdYSE, dxdYNW, dxdYNE, dydYSW, dydYSE, dydYNW, dydYNE )
nodeSW = stencil(kxxW, kxyW, kxxE, kxyE,  kyyS, kyxS, kyyN, kyxN,       kxy2, kxy1, kxy1, kxy1, kxx2, kxx1, kxx1, kxx1, kyy2, kyy1, kyy1, kyy1,    dxdXN, dxdXS, dydXW, dydXE, dxdΥN, dxdΥS, dydΥW, dydΥE,   dxdXSW, dxdXSE, dxdXNW, dxdXNE, dydXSW, dydXSE, dydXNW, dydXNE, dxdYSW, dxdYSE, dxdYNW, dxdYNE, dydYSW, dydYSE, dydYNW, dydYNE )
a=nodeNE.diff(uSW)
b=nodeSW.diff(uNE)
(a-b).simplify()

0